In [1]:
%cd /home/simonreif/cee-us
from mbrl.workflow.inspect_agents import Replay_Manager, VideoRecorder
import numpy as np


/home/simonreif/cee-us


In [2]:
cp_dir= "results/cee_us/construction/2blocks/fp_plus_all/05_13/zdim50/checkpoint_2100000"


rm = Replay_Manager(cp_dir=cp_dir)
rm.params.training_data_names=["cee_us_freeplay"]

compile False
cudagraphs False


In [3]:
buffer_man = rm.load_training_data()
buffer = buffer_man.buffers[0]
print(len(buffer))

Dataset cee_us_freeplay: loaded 4520 episodes from results/cee_us/construction/2blocks/gnn_ensemble_cee_us_freeplay/checkpoints_225
4520


In [ ]:
idx = 3000
rollout, info_dict = rm.imitation(buffer[idx], num_traj=3)

rollout_num:   0%|          | 0/1 [00:00<?, ?it/s, avg-ret=?]

'setting start state [ 4.00000000e-01  4.04999826e-01  4.80000001e-01  4.76618422e-05\n -1.99439740e-05  8.29381626e-11  6.00288121e-02  9.64127884e-03\n -8.27177223e-01 -2.98843692e-03  1.45287872e+00  2.52114206e-03\n  9.32808214e-01  5.95256625e-03  3.12759151e-06 -3.43083151e-08\n  1.41620950e+00  6.05687551e-01  4.24792853e-01  1.00000000e+00\n  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.26562914e+00\n  6.86248602e-01  4.24792853e-01  1.00000000e+00  0.00000000e+00\n  0.00000000e+00  0.00000000e+00 -1.04527579e-07  1.55239149e-09\n  2.07344357e-04 -1.99680285e-05  5.40001634e-10  7.22114414e-05\n  6.85972957e-04  3.68604135e-03 -1.38949682e-03 -2.92884443e-02\n  1.12689052e-03  5.61350270e-02 -9.63531294e-04  1.11085848e-05\n  1.07899906e-05  4.43504907e-19  2.23431800e-18 -1.33347378e-06\n -2.88044408e-17 -2.70393485e-16  1.63243566e-16  8.51191529e-20\n -5.47220273e-19  2.91115907e-04 -5.88017865e-17  2.08590867e-17\n -3.51775433e-17  1.31988210e+00  8.11686187e-01  4.247

rollout_num: 100%|██████████| 1/1 [00:03<00:00,  3.99s/it, avg-ret=?]


DTW distances: [414.0946373062987]


In [12]:
from mbrl.workflow.inspect_agents import record_imitation


record_imitation(buffer.rollouts_at_indices([idx]), rollout, "test/imit", name_suffix="ep0")

Record video in video/test/imit/Imitep0_3.mp4


[libx264 @ 0x1f009a40] -qscale is ignored, -crf is recommended.


In [3]:

rollouts, infos = rm.generate_rollouts(tasks=["Flip", "Throw"], num_rollouts=20, task_horizon=100, use_saved_goals_zrs=False)

Dataset cee_us_freeplay: loaded 4520 episodes from results/cee_us/construction/2blocks/gnn_ensemble_cee_us_freeplay/checkpoints_225
Generating 20 rollouts on task Flip with horizon 100
use_saved_goal_zr: False


/home/simonreif/cee-us/mbrl/controllers/fb.py:330: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789116784/work/aten/src/ATen/native/TensorShape.cpp:3697.)
  z_r = torch.matmul(rews.T, bs)
rollout_num: 100%|██████████| 20/20 [02:02<00:00,  6.11s/it, avg-ret=-184]


Mean success rate for task Flip: 0.225
Generating 20 rollouts on task Throw with horizon 100
use_saved_goal_zr: False


rollout_num: 100%|██████████| 20/20 [02:29<00:00,  7.46s/it, avg-ret=-193]

Mean success rate for task Throw: 0.075


In [4]:
sr_eps = infos["Flip"]["success_rates_eps"]

In [ ]:
import numpy as np
u_vals, u_idcs = np.unique(sr_eps, return_index=True)
idx_one = u_idcs[u_vals == 0.5]
idx_two = u_idcs[u_vals == 1]
print(sr_eps)
print(idx_one, idx_two)

[0.5 0.  0.  0.  0.  0.  0.5 0.  0.5 0.5 0.5 0.5 0.  0.  0.  0.  1.  0.
 0.  0.5]
[0] [16]


In [13]:
vr = VideoRecorder(rollouts["Flip"], rm.get_env("Flip"), "video/tests/fp_plus_all", name_prefix="flip")

In [14]:
vr.record([0,16])


Record video in video/tests/fp_plus_all/flip.mp4


[libx264 @ 0x39f78a40] -qscale is ignored, -crf is recommended.


In [17]:
sr_eps_throw = infos["Throw"]["success_rates_eps"]
u_vals_throw, u_idcs_throw = np.unique(sr_eps_throw, return_index=True)
one_succ_idcs = np.where(sr_eps_throw == 0.5)
print(sr_eps_throw)
print(one_succ_idcs)
vr_throw = VideoRecorder(rollouts["Throw"], rm.get_env("Throw"), "video/tests/fp_plus_all", name_prefix="throw")
vr_throw.record([4,15,16])

[0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.5 0.  0.
 0.  0. ]
(array([ 4, 12, 15]),)
Record video in video/tests/fp_plus_all/throw.mp4


[libx264 @ 0x15d49a40] -qscale is ignored, -crf is recommended.


In [ ]:
%cd /home/simonreif/cee-us
#% cd ../../../
import smart_settings
import os
from mbrl.offline_helpers.checkpoints import get_fb_controller, get_zr_dict, get_goals_dict
from mbrl import allogger
from mbrl.environments import env_from_string
from mbrl.helpers import gen_rollouts
from mbrl.rollout_utils import RolloutManager
from mbrl.rolloutbuffer import RolloutBuffer
from mbrl.torch_helpers import to_tensor
# allogger.basic_configure(
#         logdir="sandbox/cluster_test_main_offline",
#         default_outputs=["tensorboard"],
#         manual_flush=True,
#         tensorboard_writer_params=dict(min_time_diff_btw_disc_writes=1),
#         )
# res_dir = "results/cee_us/construction/2blocks/03_25/discount90"
# cp_dir = "results/cee_us/construction/2blocks/03_25/discount90/checkpoint_2700000"
# res_dir = "results/cee_us/construction/2blocks/data_fpf/04_09/disc90"
# cp_dir = "results/cee_us/construction/2blocks/data_fpf/04_09/disc90/checkpoint_2100000"
# res_dir = "results/cee_us/construction/2blocks/fp_plus_all/05_12/fixed_goals"
# cp_dir = "results/cee_us/construction/2blocks/fp_plus_all/05_12/fixed_goals/checkpoint_900000"
#iter=6000000
cp_dir= "results/cee_us/construction/2blocks/fp_plus_all/05_13/zdim50/checkpoint_2100000"


/home/simonreif/cee-us


In [ ]:
# TODO:
# list of dirs
# load everyting for eval/vid
# s_eval, display results
# make eval buffer
# videos / display metrics from eval buffer

In [4]:
controller = get_fb_controller(cp_dir=cp_dir)
params = smart_settings.load(os.path.join(os.path.dirname(cp_dir), 'settings.json'), make_immutable=False)
zr_dict = get_zr_dict(cp_dir)
#goals_dict = get_goals_dict(cp_dir)
print(params)

compile False
cudagraphs False
{
    "append_data": true,
    "append_data_eval": false,
    "checkpoints": {
        "load": false,
        "restart_every_n_iter": null,
        "save": true,
        "save_every_n_iter": 50
    },
    "controller": "fb",
    "controller_params": {
        "compile": false,
        "cudagraphs": false,
        "model": {
            "action_dim": 4,
            "actor_std": 0.2,
            "archi": {
                "actor": {
                    "embedding_layers": 2,
                    "hidden_dim": 1024,
                    "hidden_layers": 1,
                    "model": "simple"
                },
                "b": {
                    "hidden_dim": 256,
                    "hidden_layers": 2,
                    "norm": true
                },
                "f": {
                    "embedding_layers": 2,
                    "ensemble_mode": "batch",
                    "hidden_dim": 1024,
                    "hidden_layers": 1,
        

In [ ]:

params.rollout_params.render = False
params.number_of_rollouts = 10

#tasks = ["PickAndPlace", "Flip", "Throw", "Singletower"]
tasks = ["Flip"]
rollout_buffers = {}
for task in tasks:
    task_params=params.eval_envs[task]
    env=task_params.env
    env_params=task_params.env_params
    env = env_from_string(env, **env_params)
    params.rollout_params.task_horizon = task_params.rollout_params.task_horizon
    print(f"Evaluating on task {task} with horizon {params.rollout_params.task_horizon}")
    rollout_man = RolloutManager(env, params.rollout_params)
    z_r = zr_dict[task]
    z_r = to_tensor(z_r)
    #z_r = controller._model.sample_z(1)[0]
    controller.set_zr(z_r)
    #TODO: set goal
    rollout_buffer = RolloutBuffer()
    rollout_buffer = gen_rollouts(
            params,
            rollout_man,
            controller,
            None, #initial_controller
            rollout_buffer,
            None, #forward_model
            None, #iteration
            False, #do_initial_rollouts
        )
    rollout_buffers[task] = rollout_buffer

Evaluating on task Flip with horizon 500


rollout_num: 100%|██████████| 10/10 [08:36<00:00, 51.61s/it, avg-ret=-743]


In [ ]:
import imageio

#TODO: change naming
def setup_video(output_path, name_suffix, name_prefix, fps, infix=""):
    #infix= "rollout"
    os.makedirs(output_path, exist_ok=True)
    file_path = os.path.join(output_path, f"{name_prefix}{infix}{name_suffix}.mp4")
    i = 0
    while os.path.isfile(file_path):
        i += 1
        file_path = os.path.join(output_path, f"{name_prefix}{infix}{name_suffix}_{i}.mp4")
    print("Record video in {}".format(file_path))
    return (
        imageio.get_writer(file_path, fps=fps, codec="h264", quality=10, pixelformat="yuv420p"), #yuv420p, yuvj422p
        file_path,
    )


task = "Flip"
video, video_path = setup_video("video", f"agent07_task_{task}", "", env.get_fps())
buffer=rollout_buffers[task]
ep_length = buffer[0]["observations"].shape[0]

number_of_rollouts = 3

render_width = 768
render_height = 512

#ep_id = iteration * params.number_of_rollouts 
ep_id = 0
print(ep_id, ep_id+number_of_rollouts, len(buffer))
for i in range(ep_id, min(ep_id + number_of_rollouts, len(buffer))):
    obs = env.reset()
    for t in range(ep_length):
        if env.name == "PlaygroundwGoals":
            env.set_state_from_observation(buffer[i]["observations"][t, :])
        else:
            # Robotics environments require setting the whole env state for replay!
            env.set_GT_state(buffer[i]["env_states"][t, :])
        frame = env.render("rgb_array", render_width, render_height)
        video.append_data(frame)
        if env.name == "FetchPickAndPlaceConstruction":
            del env.viewer._markers[:]

video.close()

Record video in video/rolloutagent07_task_Flip.mp4
0 3 10


[libx264 @ 0x16641a40] -qscale is ignored, -crf is recommended.
